In [ ]:
#----------------------------------------------------------------------------#
#       Procesamiento de grandes volumenes de datos 2020-2                   #
#                     Proyecto 2 (Streaming)                                 #
#                       Alejandro Ayala Gil                                  #
#                       Esteban Cardona Gil                                  #
#                    Juan Camilo Gomez Muñoz                                 #
#                        Julian Paredes C                                    #
#                    Tania C. Obando Suárez                                  #
#----------------------------------------------------------------------------#


#Importando librerias
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.sql import SparkSession, DataFrameStatFunctions, DataFrameNaFunctions
from pyspark.sql.functions import *
#import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def foreach_batch_function(df, epoch_id):
    # Transform and write batchDF
    pass

def showData(spark,dataframe,features):
    n_features = len(features)
    fields = ""
    for i in range(n_features-1):
        fields+=features[i]+","
    fields+=features[-1]
    dataframe.createOrReplaceTempView("table1")
    dataframe = spark.sql("SELECT {} from table1".format(fields))
    return dataframe

def categoricalToNumerical(df):

    #Reemplazar valores categoricos a númericos
    df = df.withColumn("school", regexp_replace("school", "GP", "0"))
    df = df.withColumn("school", regexp_replace("school", "MS", "1"))
    df = df.withColumn("sex", regexp_replace("sex", "F", "0"))
    df = df.withColumn("sex", regexp_replace("sex", "M", "1"))
    df = df.withColumn("address", regexp_replace("address", "R", "0"))
    df = df.withColumn("address", regexp_replace("address", "U", "1"))
    df = df.withColumn("famsize", regexp_replace("famsize", "LE3", "0"))
    df = df.withColumn("famsize", regexp_replace("famsize", "GT3", "1"))
    df = df.withColumn("Pstatus", regexp_replace("Pstatus", "A", "0"))
    df = df.withColumn("Pstatus", regexp_replace("Pstatus", "T", "1"))
    df = df.withColumn("Mjob", regexp_replace("Mjob", "other", "0"))
    df = df.withColumn("Mjob", regexp_replace("Mjob", "at_home", "1"))
    df = df.withColumn("Mjob", regexp_replace("Mjob", "teacher", "2"))
    df = df.withColumn("Mjob", regexp_replace("Mjob", "services", "3"))
    df = df.withColumn("Mjob", regexp_replace("Mjob", "health", "4"))
    df = df.withColumn("Fjob", regexp_replace("Fjob", "other", "0"))
    df = df.withColumn("Fjob", regexp_replace("Fjob", "at_home", "1"))
    df = df.withColumn("Fjob", regexp_replace("Fjob", "teacher", "2"))
    df = df.withColumn("Fjob", regexp_replace("Fjob", "services", "3"))
    df = df.withColumn("Fjob", regexp_replace("Fjob", "health", "4"))
    df = df.withColumn("reason", regexp_replace("reason", "other", "0"))
    df = df.withColumn("reason", regexp_replace("reason", "home", "1"))
    df = df.withColumn("reason", regexp_replace("reason", "reputation", "2"))
    df = df.withColumn("reason", regexp_replace("reason", "course", "3"))
    df = df.withColumn("guardian", regexp_replace("guardian", "father", "1"))
    df = df.withColumn("guardian", regexp_replace("guardian", "mother", "2"))
    df = df.withColumn("guardian", regexp_replace("guardian", "other", "0"))
    df = df.withColumn("schoolsup", regexp_replace("schoolsup", "no", "0"))
    df = df.withColumn("schoolsup", regexp_replace("schoolsup", "yes", "1"))
    df = df.withColumn("famsup", regexp_replace("famsup", "no", "0"))
    df = df.withColumn("famsup", regexp_replace("famsup", "yes", "1"))
    df = df.withColumn("paid", regexp_replace("paid", "no", "0"))
    df = df.withColumn("paid", regexp_replace("paid", "yes", "1"))
    df = df.withColumn("activities", regexp_replace("activities", "no", "0"))
    df = df.withColumn("activities", regexp_replace("activities", "yes", "1"))
    df = df.withColumn("nursery", regexp_replace("nursery", "no", "0"))
    df = df.withColumn("nursery", regexp_replace("nursery", "yes", "1"))
    df = df.withColumn("higher", regexp_replace("higher", "no", "0"))
    df = df.withColumn("higher", regexp_replace("higher", "yes", "1"))
    df = df.withColumn("internet", regexp_replace("internet", "no", "0"))
    df = df.withColumn("internet", regexp_replace("internet", "yes", "1"))
    df = df.withColumn("romantic", regexp_replace("romantic", "no", "0"))
    df = df.withColumn("romantic", regexp_replace("romantic", "yes", "1"))
    #df.show()
    return(df)

#Encabezado del dataframe
headings = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob',
         'Fjob','reason','guardian','traveltime','studytime','failures','schoolsup',
         'famsup','paid','activities','nursery','higher','internet','romantic',
         'famrel','freetime','goout','Dalc','Walc','health','absences','G1','G2','G3']
    
spark = SparkSession.builder.appName("Student").getOrCreate()
#Crear dataframe
esquema=spark.read.csv('student-por.csv',sep=';',header=True).schema
df=spark.readStream.csv('Datos streaming/read',sep=';',header=True,schema=esquema)

#Reemplazar valores categoricos a numericos
df=categoricalToNumerical(df)
show = showData(spark,df,["school","sex"])
show = show.writeStream.format("console").outputMode("append").trigger(processingTime='2 seconds').start()
show.awaitTermination()